# 0. CNN학습하기

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision #예제 이미지 데이터셋이 모여있는 모듈
from torchvision import datasets, transforms

import sklearn
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
#rand 함수를 쓰는데 seed를 고정해서 쓴다. -> 평가 시 변인통제를 위해
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSHEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

#강의랑 똑같은 결과를 내라고 랜덤값 고정하는데 하지말자...
#seed_everything(42)

# 1. Convolutional Layer

In [2]:
#conv 레이어 정의하기
conv = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding='same')

for p in conv.parameters():
    print(p)

Parameter containing:
tensor([[[[ 0.0280,  0.0392, -0.1059],
          [-0.0659, -0.0331, -0.0181],
          [-0.0187, -0.1907,  0.1569]],

         [[ 0.1831,  0.1193, -0.1420],
          [ 0.0094, -0.0953, -0.0069],
          [-0.0298, -0.1232, -0.1739]],

         [[ 0.0973,  0.1780,  0.1132],
          [ 0.1321, -0.1460, -0.0755],
          [-0.0322,  0.1486,  0.0478]]],


        [[[-0.0800,  0.0025,  0.0167],
          [ 0.0921, -0.1618,  0.1182],
          [-0.0650, -0.0144, -0.0281]],

         [[ 0.1338,  0.1634, -0.0177],
          [-0.0260, -0.0130,  0.1669],
          [ 0.0511, -0.0106,  0.1660]],

         [[ 0.1558, -0.1395, -0.0070],
          [-0.0973,  0.0927,  0.1091],
          [ 0.0555,  0.0011,  0.1365]]]], requires_grad=True)
Parameter containing:
tensor([-0.0401, -0.0243], requires_grad=True)


In [3]:
#input 텐서 정의하기
input = torch.randn(10, 3, 7, 7) #배치사이즈, input_dim, (세로 가로)-> 이미지 크기

#정의한 input텐서에 conv를 통과시켜 보기
output = conv(input)

print(output.shape)
#여기까지가 진짜 간단히 알아본 conv 작동예제

torch.Size([10, 2, 7, 7])


# 2. CNN 설계하기

In [6]:
#데이터셋 받아오고 전처리 수행하기

train_loader = torch.utils.data.DataLoader( #여기서 train=True는 훈련용 데이터 받아라
    datasets.FashionMNIST("data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(), #download=True는 파일이 없으면 다운받아라
        transforms.Normalize((0.1307, ), (0.3081, )) #transform는 이미지 전처리의 방법론
    ])), #Normalize는 정규분포로 스케일링 작업 (평균, 분산)
    batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader( #여기서 train=False는 평가용 데이터 받아라
    datasets.FashionMNIST("data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307, ), (0.3081, ))
    ])),
    batch_size=64, shuffle=True
)

In [7]:
#CNN 모델 설계하기

class Fashion_MNIST_Model(nn.Module):
    def __init__(self):
        super(Fashion_MNIST_Model, self).__init__()

        #conv1의 input_dim이 1인 이유 : Fashion_MNIST_Model의 데이터셋은 '그레이스케일'이다.
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3) #input_dim이 1, output_dim은 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3) #길쭉한 정육면체로 만드는 과정
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = F.max_pool2d(x, 2) #바로 위 레이어를 반으로 줄임

        x = self.conv2(x)
        x = F.relu(x)

        x = F.max_pool2d(x, 2)

        x = x.view(-1, 64 * 5 * 5) #Flatten과정 -> 
        #64는 conv2를 통과하고 나온 채널 수, 5x5는 conv2를 통과하고 나온 feature map사이즈

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)

        x = F.log_softmax(x, dim=1)

        return x
    
CNN_ex_model = Fashion_MNIST_Model()

In [8]:
#로스함수, 옵티마이저 선언
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_ex_model.parameters(), lr=0.001)

In [10]:
#훈련 시작!

CNN_ex_model.train()

for epoch in tqdm(range(5)): #CPU에서 훈련시키는거라 속도가 엄청오래걸리니 5개만 함..
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 1, 28, 28)#이미지의 채널, 사이즈(y, x)
        labels = labels

        #여기가 forward
        outputs = CNN_ex_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad() #옵티마이저 초기화

        #여기가 backward
        loss.backward()
        optimizer.step() #모델 파라미터 업데이트


        #여기는 중간중간 확인과정
        if (i+1) % 100 == 0: #100스탭당 한번씩 프린트하기
            print('Epoch : [{}/{}], step : [{}/{}], Loss : {:.4f}'.format(
                epoch+1, 5, i+1, len(train_loader), loss.item()
            ))


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch : [1/5], step : [100/938], Loss : 0.0407
Epoch : [1/5], step : [200/938], Loss : 0.1118
Epoch : [1/5], step : [300/938], Loss : 0.2418
Epoch : [1/5], step : [400/938], Loss : 0.1467
Epoch : [1/5], step : [500/938], Loss : 0.1138
Epoch : [1/5], step : [600/938], Loss : 0.2056
Epoch : [1/5], step : [700/938], Loss : 0.1511
Epoch : [1/5], step : [800/938], Loss : 0.1053
Epoch : [1/5], step : [900/938], Loss : 0.2325


 20%|██        | 1/5 [01:04<04:18, 64.64s/it]

Epoch : [2/5], step : [100/938], Loss : 0.1791
Epoch : [2/5], step : [200/938], Loss : 0.0800
Epoch : [2/5], step : [300/938], Loss : 0.2170
Epoch : [2/5], step : [400/938], Loss : 0.1608
Epoch : [2/5], step : [500/938], Loss : 0.0785
Epoch : [2/5], step : [600/938], Loss : 0.2059
Epoch : [2/5], step : [700/938], Loss : 0.1331
Epoch : [2/5], step : [800/938], Loss : 0.1057
Epoch : [2/5], step : [900/938], Loss : 0.1782


 40%|████      | 2/5 [02:02<03:01, 60.41s/it]

...

 60%|██████    | 3/5 [03:00<01:58, 59.34s/it]

Epoch : [4/5], step : [100/938], Loss : 0.1557
Epoch : [4/5], step : [200/938], Loss : 0.1078
Epoch : [4/5], step : [300/938], Loss : 0.1509
Epoch : [4/5], step : [400/938], Loss : 0.1641
Epoch : [4/5], step : [500/938], Loss : 0.1545
Epoch : [4/5], step : [600/938], Loss : 0.0585
Epoch : [4/5], step : [700/938], Loss : 0.0828
Epoch : [4/5], step : [800/938], Loss : 0.1677
Epoch : [4/5], step : [900/938], Loss : 0.1492


 80%|████████  | 4/5 [03:57<00:58, 58.66s/it]

Epoch : [5/5], step : [100/938], Loss : 0.0894
Epoch : [5/5], step : [200/938], Loss : 0.1592
Epoch : [5/5], step : [300/938], Loss : 0.1274
Epoch : [5/5], step : [400/938], Loss : 0.0444
Epoch : [5/5], step : [500/938], Loss : 0.0704
Epoch : [5/5], step : [600/938], Loss : 0.1202
Epoch : [5/5], step : [700/938], Loss : 0.1110
Epoch : [5/5], step : [800/938], Loss : 0.0820
Epoch : [5/5], step : [900/938], Loss : 0.0560


100%|██████████| 5/5 [04:55<00:00, 59.14s/it]


# GPU에서 구동시키기

In [15]:
#GPU사용 가능 확인하기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#학습 모델 GPU로 이전하기
print(CNN_ex_model.to(device))

print(next(CNN_ex_model.parameters()).device) #모델 파라미터의 위치 확인

cuda
Fashion_MNIST_Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
cuda:0


In [16]:
#로스 함수랑 옵티아이저 재 선언해서 GPU로 이전하기
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_ex_model.parameters(), lr=0.001)

In [17]:
#훈련 시작(GPU모드에서)

for epoch in tqdm(range(100)): #이번엔 GPU에서 훈련시키니 자
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)

        #여기가 forward
        outputs = CNN_ex_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad() #옵티마이저 초기화

        #여기가 backward
        loss.backward()
        optimizer.step() #모델 파라미터 업데이트


        #여기는 중간중간 확인과정
        if (epoch+1) % 10 == 0 and (i+1) % 100 == 0: #100스탭당 한번씩 프린트하기
            print('Epoch : [{}/{}], step : [{}/{}], Loss : {:.4f}'.format(
                epoch+1, 100, i+1, len(train_loader), loss.item()
            ))

  9%|▉         | 9/100 [01:59<18:46, 12.38s/it]

Epoch : [10/5], step : [100/938], Loss : 0.0138
Epoch : [10/5], step : [200/938], Loss : 0.0153
Epoch : [10/5], step : [300/938], Loss : 0.0394
Epoch : [10/5], step : [400/938], Loss : 0.0370
Epoch : [10/5], step : [500/938], Loss : 0.0857
Epoch : [10/5], step : [600/938], Loss : 0.0038
Epoch : [10/5], step : [700/938], Loss : 0.0930
Epoch : [10/5], step : [800/938], Loss : 0.0656
Epoch : [10/5], step : [900/938], Loss : 0.0406


 19%|█▉        | 19/100 [04:00<16:26, 12.18s/it]

Epoch : [20/5], step : [100/938], Loss : 0.0153
Epoch : [20/5], step : [200/938], Loss : 0.0258
Epoch : [20/5], step : [300/938], Loss : 0.0097
Epoch : [20/5], step : [400/938], Loss : 0.0190
Epoch : [20/5], step : [500/938], Loss : 0.0197
Epoch : [20/5], step : [600/938], Loss : 0.0152
Epoch : [20/5], step : [700/938], Loss : 0.0117
Epoch : [20/5], step : [800/938], Loss : 0.0020
Epoch : [20/5], step : [900/938], Loss : 0.1258


 29%|██▉       | 29/100 [06:02<14:24, 12.17s/it]

...

 89%|████████▉ | 89/100 [18:42<02:24, 13.18s/it]

Epoch : [90/5], step : [100/938], Loss : 0.0000
Epoch : [90/5], step : [200/938], Loss : 0.0048
Epoch : [90/5], step : [300/938], Loss : 0.0004
Epoch : [90/5], step : [400/938], Loss : 0.0000
Epoch : [90/5], step : [500/938], Loss : 0.0000
Epoch : [90/5], step : [600/938], Loss : 0.0007
Epoch : [90/5], step : [700/938], Loss : 0.0000
Epoch : [90/5], step : [800/938], Loss : 0.0006
Epoch : [90/5], step : [900/938], Loss : 0.0025


 99%|█████████▉| 99/100 [20:54<00:13, 13.16s/it]

Epoch : [100/5], step : [100/938], Loss : 0.0000
Epoch : [100/5], step : [200/938], Loss : 0.0000
Epoch : [100/5], step : [300/938], Loss : 0.0000
Epoch : [100/5], step : [400/938], Loss : 0.0351
Epoch : [100/5], step : [500/938], Loss : 0.0071
Epoch : [100/5], step : [600/938], Loss : 0.0000
Epoch : [100/5], step : [700/938], Loss : 0.0001
Epoch : [100/5], step : [800/938], Loss : 0.0066
Epoch : [100/5], step : [900/938], Loss : 0.0000


100%|██████████| 100/100 [21:07<00:00, 12.68s/it]


# 학습된 모델 평가하기

In [22]:
#모델을 평가모드로 전환
CNN_ex_model.eval()

#평가를 위해 임의의 두 변수를 선언후 GPU로 보냄
Y_pred, Y_true = torch.zeros([64]).to(device), torch.zeros([64]).to(device)
total, correct = 0, 0


with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 1, 28, 28).to(device) #평가용 이미지를 GPU로 이전
        labels = labels.to(device)

        outputs = CNN_ex_model(images) #softmax를 통과하니 확률로 값이 나옴
        _, predicted = torch.max(outputs.data, 1)


        if(predicted.shape[0] != 64): #이거는 배치 이미지 부족할 때 동작하는 함수
            predicted = torch.cat([predicted, torch.zeros(64-predicted.shape[0]).to(device)])
            labels = torch.cat([labels, torch.zeros(64-labels.shape[0]).to(device)])

        
        Y_pred = torch.cat([Y_pred, predicted])
        Y_true = torch.cat([Y_true, labels])

        total = total + labels.size(0)
        correct = correct + (predicted == labels).sum().item()

print("수행 결과 정확도 : {:.2f}%".format((correct/total)*100))

수행 결과 정확도 : 90.96%


In [20]:
#이제 수행한 결과를 리포트로 출력하는데 CPU로 내려야 한다.
Y_true = Y_true.detach().cpu().numpy()
Y_pred = Y_pred.detach().cpu().numpy()

print(classification_report(Y_true, Y_pred))


              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86      1112
         1.0       0.99      0.98      0.99      1000
         2.0       0.85      0.87      0.86      1000
         3.0       0.91      0.91      0.91      1000
         4.0       0.88      0.85      0.86      1000
         5.0       0.98      0.98      0.98      1000
         6.0       0.77      0.74      0.75      1000
         7.0       0.96      0.96      0.96      1000
         8.0       0.98      0.98      0.98      1000
         9.0       0.97      0.96      0.97      1000

    accuracy                           0.91     10112
   macro avg       0.91      0.91      0.91     10112
weighted avg       0.91      0.91      0.91     10112

